In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re

In [ ]:
feature_ID_list = pd.read_csv('./Feature ID list for shock(MIMIC-4).csv')
feature_name = feature_ID_list['Feature name'].tolist()
feature_ID_list.set_index('Feature name', inplace=True)

inputevents = pd.read_csv('./inputevents.csv')
patients = pd.read_csv('./icustays.csv')

inputevents = pd.merge(inputevents, patients, on=['subject_id','hadm_id','stay_id'], how='left')
inputevents.reset_index(drop=True, inplace=True)

condition1= (inputevents['intime'] <= inputevents['starttime'])
condition2 = (inputevents['outtime'] >= inputevents['starttime'])

inputevents = inputevents[condition1 & condition2]
inputevents = inputevents.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
inputevents.reset_index(drop=True, inplace=True)

Transfusion

In [ ]:
transfusion_id_ls =[225168,226368,226370,227070]

Transfusion = inputevents[inputevents['itemid'].isin(transfusion_id_ls)]
Transfusion.reset_index(drop=True, inplace=True)
Transfusion = Transfusion[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]

Transfusion = Transfusion[Transfusion['amount'].notna()]
condition1 = Transfusion['amount']>=0 
Transfusion = Transfusion[condition1]
Transfusion.reset_index(drop=True, inplace=True)
Transfusion = Transfusion.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
Transfusion = Transfusion[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
Transfusion.reset_index(drop=True, inplace=True)

Transfusion['starttime'] = pd.to_datetime(Transfusion['starttime'])
Transfusion['endtime'] = pd.to_datetime(Transfusion['endtime'])
Transfusion['outtime'] = pd.to_datetime(Transfusion['outtime'])
Transfusion['intime'] = pd.to_datetime(Transfusion['intime'])

Transfusion = Transfusion[Transfusion['starttime']<=Transfusion['endtime']]
print(Transfusion.shape)
Transfusion.reset_index(drop=True, inplace=True)

def resampling_func2(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', 'Transfusion']

    x = data.reset_index(drop=True)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df['Transfusion'] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df['Transfusion'][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df['Transfusion'][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df['Transfusion'][i] = sa1_sum

                else :
                    new_df['Transfusion'][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df['Transfusion'][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['Transfusion'][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df['Transfusion'][i] = sa2_sum 

                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df['Transfusion'][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df['Transfusion'][i] = sa1_sum
                        else :  
                            new_df['Transfusion'][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', 'Transfusion']]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_Transfusion = Transfusion.groupby('stay_id').apply(resampling_func2)
result_Transfusion.reset_index(drop=True, inplace=True)
print('---------------------------------------------------------------------------------')
result_Transfusion.to_csv('C:/Users/DAHS/Desktop/KU X PITTS/Data preprocessed 4/10min Transfusion.csv', index=False)

print(result_Transfusion['stay_id'].unique().shape)

IV Fluids

In [ ]:
crystalloids_id_ls = feature_ID_list['inputevents'][feature_name[18]]
crystalloids_id_ls = [int(x) for x in crystalloids_id_ls.split(',')]

colloids_id_ls = feature_ID_list['inputevents'][feature_name[19]]
colloids_id_ls = [int(x) for x in colloids_id_ls.split(',')]

In [ ]:
crystalloids = inputevents[inputevents['itemid'].isin(crystalloids_id_ls)]
colloids = inputevents[inputevents['itemid'].isin(colloids_id_ls)]

In [ ]:
crystalloids.reset_index(drop=True, inplace=True)

colloids.reset_index(drop=True, inplace=True)

In [ ]:
IV_fluids = pd.concat([crystalloids,colloids], axis=0)
IV_fluids = IV_fluids.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
IV_fluids.reset_index(drop=True, inplace=True)
IV_fluids = IV_fluids[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]

In [ ]:
crystalloids=0
colloids=0

In [ ]:
IV_fluids = IV_fluids[IV_fluids['amount'].notna()]
condition3 = IV_fluids['amount']<10000 
condition1 = IV_fluids['amount']>=0 
IV_fluids = IV_fluids[(condition1) & (condition3)]
IV_fluids.reset_index(drop=True, inplace=True)
IV_fluids = IV_fluids.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
IV_fluids = IV_fluids[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
IV_fluids.reset_index(drop=True, inplace=True)

In [ ]:
IV_fluids['starttime'] = pd.to_datetime(IV_fluids['starttime'])
IV_fluids['endtime'] = pd.to_datetime(IV_fluids['endtime'])
IV_fluids['outtime'] = pd.to_datetime(IV_fluids['outtime'])
IV_fluids['intime'] = pd.to_datetime(IV_fluids['intime'])

IV_fluids = IV_fluids[IV_fluids['starttime']<=IV_fluids['endtime']]
print(IV_fluids.shape)
IV_fluids.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func2(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', 'IV fluids']

    x = data.reset_index(drop=True)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df['IV fluids'] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df['IV fluids'][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df['IV fluids'][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df['IV fluids'][i] = sa1_sum

                else :
                    new_df['IV fluids'][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df['IV fluids'][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['IV fluids'][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df['IV fluids'][i] = sa2_sum 

                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df['IV fluids'][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df['IV fluids'][i] = sa1_sum
                        else :  
                            new_df['IV fluids'][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', 'IV fluids']]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_IV_fluids = IV_fluids.groupby('stay_id').apply(resampling_func2)
result_IV_fluids.reset_index(drop=True, inplace=True)
print('---------------------------------------------------------------------------------')
result_IV_fluids.to_csv('./10min Fluids.csv', index=False)

In [ ]:
print(result_IV_fluids['stay_id'].unique().shape)

Vasopressors

In [ ]:
feature_ID_list = pd.read_csv('./Feature ID list for shock(MIMIC-4).csv')
feature_name = feature_ID_list['Feature name'].tolist()
feature_ID_list.set_index('Feature name', inplace=True)

In [ ]:
vasopressors_id_ls = feature_ID_list['inputevents'][feature_name[22]]
vasopressors_id_ls = [int(x) for x in vasopressors_id_ls.split(',')]

In [ ]:
vasopressors = inputevents[inputevents['itemid'].isin(vasopressors_id_ls)]
vasopressin2_index = vasopressors[vasopressors['itemid']==222315].index
vasopressors['rate'][vasopressin2_index] = vasopressors['rate'][vasopressin2_index]*2.5/60
phenylephrine2_index = vasopressors[vasopressors['itemid']==221749].index
vasopressors['rate'][phenylephrine2_index] = vasopressors['rate'][phenylephrine2_index]*0.1
condition3 = vasopressors['rate']<=10
condition1 = vasopressors['rate']>=0
vasopressors = vasopressors[(condition1)&(condition3)]
vasopressors = vasopressors[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'rate']]
vasopressors.reset_index(drop=True, inplace=True)

In [ ]:
vasopressors['starttime'] = pd.to_datetime(vasopressors['starttime'])
vasopressors['endtime'] = pd.to_datetime(vasopressors['endtime'])
vasopressors['outtime'] = pd.to_datetime(vasopressors['outtime'])
vasopressors['intime'] = pd.to_datetime(vasopressors['intime'])

vasopressors = vasopressors[vasopressors['starttime']<=vasopressors['endtime']]
print(vasopressors.shape)
vasopressors.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func_vaso(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', 'Vasopressors']

    x = data.reset_index(drop=True)
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0],x['outtime'][0],freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df['Vasopressors'] = 0
    
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = date_range[i]>=x['starttime']
            sa = x[condition1]
            if len(sa)!=0:
                new_df['Vasopressors'][i] = sa['rate'].max()
            else :
                new_df['Vasopressors'][i] = 0

        else :
            condition2 = date_range[i-1]<x['starttime']
            condition3 = date_range[i]>=x['starttime']
            condition4 = date_range[i-1]<x['endtime']
            condition5 = date_range[i]>=x['endtime']
            sa = x[((condition2)&(condition3)) | ((condition4)&(condition5))]
            if len(sa)!=0:
                new_df['Vasopressors'][i] = sa['rate'].max()
            else :
                new_df['Vasopressors'][i] = 0


    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', 'Vasopressors']]
    new_df = new_df.reset_index(drop=True)
    
    return new_df

result_vasopressors = vasopressors.groupby('stay_id').apply(resampling_func_vaso)
result_vasopressors = result_vasopressors.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_vasopressors.to_csv('./10min Vasopressors.csv', index=False)


Epinephrine

In [ ]:
Epinephrine = inputevents[inputevents['itemid'].isin([221289,229617])]
condition1 = Epinephrine['rate']>=0
Epinephrine = Epinephrine[(condition1)]
Epinephrine = Epinephrine[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'rate']]
Epinephrine.reset_index(drop=True, inplace=True)

Epinephrine['starttime'] = pd.to_datetime(Epinephrine['starttime'])
Epinephrine['endtime'] = pd.to_datetime(Epinephrine['endtime'])
Epinephrine['outtime'] = pd.to_datetime(Epinephrine['outtime'])
Epinephrine['intime'] = pd.to_datetime(Epinephrine['intime'])

Epinephrine = Epinephrine[Epinephrine['starttime']<=Epinephrine['endtime']]
print(Epinephrine.shape)
Epinephrine.reset_index(drop=True, inplace=True)

In [ ]:
feature_name = 'Epinephrine'

def resampling_func_vaso(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]

    x = data.reset_index(drop=True)
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0],x['outtime'][0],freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = date_range[i]>=x['starttime']
            sa = x[condition1]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0

        else :
            condition2 = date_range[i-1]<x['starttime']
            condition3 = date_range[i]>=x['starttime']
            condition4 = date_range[i-1]<x['endtime']
            condition5 = date_range[i]>=x['endtime']
            sa = x[((condition2)&(condition3)) | ((condition4)&(condition5))]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0


    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
    
    return new_df

result_Epinephrine = Epinephrine.groupby('stay_id').apply(resampling_func_vaso)
result_Epinephrine = result_Epinephrine.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_Epinephrine.to_csv(f'./10min {feature_name}.csv', index=False)


Norepinephrine

In [ ]:
Norepinephrine = inputevents[inputevents['itemid']==221906]
condition1 = Norepinephrine['rate']>=0
Norepinephrine = Norepinephrine[(condition1)]
Norepinephrine = Norepinephrine[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'rate']]
Norepinephrine.reset_index(drop=True, inplace=True)

Norepinephrine['starttime'] = pd.to_datetime(Norepinephrine['starttime'])
Norepinephrine['endtime'] = pd.to_datetime(Norepinephrine['endtime'])
Norepinephrine['outtime'] = pd.to_datetime(Norepinephrine['outtime'])
Norepinephrine['intime'] = pd.to_datetime(Norepinephrine['intime'])

Norepinephrine = Norepinephrine[Norepinephrine['starttime']<=Norepinephrine['endtime']]
print(Norepinephrine.shape)
Norepinephrine.reset_index(drop=True, inplace=True)

In [ ]:
feature_name = 'Norepinephrine'

def resampling_func_vaso(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]

    x = data.reset_index(drop=True)
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0],x['outtime'][0],freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = date_range[i]>=x['starttime']
            sa = x[condition1]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0

        else :
            condition2 = date_range[i-1]<x['starttime']
            condition3 = date_range[i]>=x['starttime']
            condition4 = date_range[i-1]<x['endtime']
            condition5 = date_range[i]>=x['endtime']
            sa = x[((condition2)&(condition3)) | ((condition4)&(condition5))]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0


    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
    
    return new_df

result_Norepinephrine = Norepinephrine.groupby('stay_id').apply(resampling_func_vaso)
result_Norepinephrine = result_Norepinephrine.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_Norepinephrine.to_csv(f'./10min {feature_name}.csv', index=False)


Phenylephrine

In [ ]:
Phenylephrine = inputevents[inputevents['itemid'].isin([221749, 229630, 229631, 229632])]
condition1 = Phenylephrine['rate']>=0
Phenylephrine = Phenylephrine[(condition1)]
Phenylephrine = Phenylephrine[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'rate']]
Phenylephrine.reset_index(drop=True, inplace=True)

Phenylephrine['starttime'] = pd.to_datetime(Phenylephrine['starttime'])
Phenylephrine['endtime'] = pd.to_datetime(Phenylephrine['endtime'])
Phenylephrine['outtime'] = pd.to_datetime(Phenylephrine['outtime'])
Phenylephrine['intime'] = pd.to_datetime(Phenylephrine['intime'])

Phenylephrine = Phenylephrine[Phenylephrine['starttime']<=Phenylephrine['endtime']]
print(Phenylephrine.shape)
Phenylephrine.reset_index(drop=True, inplace=True)

In [ ]:
feature_name = 'Phenylephrine'

def resampling_func_vaso(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]

    x = data.reset_index(drop=True)
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0],x['outtime'][0],freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = date_range[i]>=x['starttime']
            sa = x[condition1]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0

        else :
            condition2 = date_range[i-1]<x['starttime']
            condition3 = date_range[i]>=x['starttime']
            condition4 = date_range[i-1]<x['endtime']
            condition5 = date_range[i]>=x['endtime']
            sa = x[((condition2)&(condition3)) | ((condition4)&(condition5))]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0


    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
    
    return new_df

result_Phenylephrine = Phenylephrine.groupby('stay_id').apply(resampling_func_vaso)
result_Phenylephrine = result_Phenylephrine.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_Phenylephrine.to_csv(f'./10min {feature_name}.csv', index=False)


Vasopressin

In [ ]:
Vasopressin = inputevents[inputevents['itemid']==222315]
condition1 = Vasopressin['rate']>=0
Vasopressin = Vasopressin[(condition1)]
Vasopressin = Vasopressin[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'rate']]
Vasopressin.reset_index(drop=True, inplace=True)

Vasopressin['starttime'] = pd.to_datetime(Vasopressin['starttime'])
Vasopressin['endtime'] = pd.to_datetime(Vasopressin['endtime'])
Vasopressin['outtime'] = pd.to_datetime(Vasopressin['outtime'])
Vasopressin['intime'] = pd.to_datetime(Vasopressin['intime'])

Vasopressin = Vasopressin[Vasopressin['starttime']<=Vasopressin['endtime']]
print(Vasopressin.shape)
Vasopressin.reset_index(drop=True, inplace=True)

In [ ]:
feature_name = 'Vasopressin'

def resampling_func_vaso(data):
    column = ['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]

    x = data.reset_index(drop=True)
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0],x['outtime'][0],freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = date_range[i]>=x['starttime']
            sa = x[condition1]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0

        else :
            condition2 = date_range[i-1]<x['starttime']
            condition3 = date_range[i]>=x['starttime']
            condition4 = date_range[i-1]<x['endtime']
            condition5 = date_range[i]>=x['endtime']
            sa = x[((condition2)&(condition3)) | ((condition4)&(condition5))]
            if len(sa)!=0:
                new_df[feature_name][i] = sa['rate'].max()
            else :
                new_df[feature_name][i] = 0


    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
    
    return new_df

result_Vasopressin = Vasopressin.groupby('stay_id').apply(resampling_func_vaso)
result_Vasopressin = result_Vasopressin.reset_index(drop=True)
print('---------------------------------------------------------------------------------')
result_Vasopressin.to_csv(f'./10min {feature_name}.csv', index=False)
